In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import re

In [2]:
df = pd.read_csv('tennis_articles_v4.csv')
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [3]:
# split the the text in the articles into sentences
sentences = []
for s in df['article_text']:
      sentences.append(sent_tokenize(s))  

In [4]:
sentences = [y for x in sentences for y in x]
sentences

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at all.

In [5]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences

['maria sharapova basically friends tennis players wta tour',
 'russian player problems openly speaking recent interview said really hide feelings much',
 'think everyone knows job',
 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match',
 'pretty competitive girl',
 'say hellos sending players flowers well',
 'uhm really friendly close many players',
 'lot friends away courts',
 'said really close lot players something strategic',
 'different men tour women tour',
 '',
 'think sport mean friends everyone categorized tennis player going get along tennis players',
 'think every person different interests',
 'friends completely different jobs interests met different parts life',
 'think everyone thinks tennis players greatest friends',
 'ultimately tennis small part',
 'many things interested',
 'basel switzerland ap roger federer advanced th swiss indoors final career beati

In [7]:
# Extract word vectors
word_embeddings = {}
f = open("glove.6B.100d.txt", encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [9]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [10]:
sim_mat = np.zeros([len(sentences), len(sentences)])
len(sentences)

119

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [14]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [15]:
nx_graph

In [16]:
scores

{0: 0.008072651798961179,
 1: 0.008501993335021196,
 2: 0.007811931837569203,
 3: 0.009293791257921525,
 4: 0.007500319324763975,
 5: 0.008146814787910128,
 6: 0.008477413394490605,
 7: 0.008251000761059316,
 8: 0.008596957854106375,
 9: 0.008257144237861842,
 10: 0.0012695751770095795,
 11: 0.008860552412326807,
 12: 0.008083543267829991,
 13: 0.008156804638877553,
 14: 0.008443316929757968,
 15: 0.008556893078356798,
 16: 0.007812826607847007,
 17: 0.008071958154050402,
 18: 0.008406021044414212,
 19: 0.008847892203456978,
 20: 0.008860865233162308,
 21: 0.00742191707978439,
 22: 0.008223434024311074,
 23: 0.008991766363083097,
 24: 0.0084639704291461,
 25: 0.006701898131178248,
 26: 0.008232471607935884,
 27: 0.00891313559514476,
 28: 0.009061682991146208,
 29: 0.009093905705613201,
 30: 0.009244521596350684,
 31: 0.008994323943841288,
 32: 0.007236869127489739,
 33: 0.008709093035113788,
 34: 0.008919130522571695,
 35: 0.009097421402414521,
 36: 0.007715970765035917,
 37: 0.0088834

In [17]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 